In [17]:
!pip install tensorflowjs #model 파일 변환을 위한 tenserflowjs 설치

     |████████████████████████████████| 77 kB 2.0 MB/s 


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import feature_column
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from sklearn.preprocessing import MultiLabelBinarizer

import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from google.colab import files  #csv 파일 업로드를 위함

In [3]:
seed = 10
np.random.seed(seed)

csvFile = files.upload() #데이터 파일 업로드

Saving pushup_dataset.csv to pushup_dataset.csv


In [5]:
import io
data = pd.read_csv(io.BytesIO(csvFile['pushup_dataset.csv']))
print(data.shape)

(2951, 55)


In [6]:
label = data['label']
features = data.drop(['label'],axis=1)

In [7]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(features, label, test_size=0.2)

In [19]:
def build_model():
  model = keras.Sequential([
                            keras.Input(shape = [len(train_x.keys())]),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(3, activation = 'softmax')
  ])

  model.compile(
      loss='categorical_crossentropy',
      optimizer = 'adam',
      metrics=['acc'])
  return model

In [20]:
model = build_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                3520      
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 3)                 195       
                                                                 
Total params: 12,035
Trainable params: 12,035
Non-trainable params: 0
_________________________________________________________________


In [21]:
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(label.str.split(","))

In [22]:
train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size=0.2)

In [23]:
from sklearn.utils import validation
model.fit(train_x,train_y, epochs = 10)

Epoch 1/10
74/74 [==============================] - 2s 5ms/step - loss: 4.0817 - acc: 0.6869
Epoch 2/10
74/74 [==============================] - 0s 5ms/step - loss: 0.6696 - acc: 0.8424
Epoch 3/10
74/74 [==============================] - 0s 5ms/step - loss: 0.4899 - acc: 0.8691
Epoch 4/10
74/74 [==============================] - 0s 5ms/step - loss: 0.3743 - acc: 0.8945
Epoch 5/10
74/74 [==============================] - 0s 7ms/step - loss: 0.3469 - acc: 0.8924
Epoch 6/10
74/74 [==============================] - 1s 7ms/step - loss: 0.2762 - acc: 0.9186
Epoch 7/10
74/74 [==============================] - 0s 5ms/step - loss: 0.2569 - acc: 0.9199
Epoch 8/10
74/74 [==============================] - 0s 4ms/step - loss: 0.2752 - acc: 0.9089
Epoch 9/10
74/74 [==============================] - 0s 4ms/step - loss: 0.2786 - acc: 0.9064
Epoch 10/10
74/74 [==============================] - 0s 4ms/step - loss: 0.1802 - acc: 0.9377


In [24]:
model.evaluate(test_x, test_y) #테스트 데이터로 검증

19/19 [==============================] - 0s 2ms/step - loss: 0.3869 - acc: 0.9002


[0.38688626885414124, 0.9001691937446594]

In [14]:
# 테스트를 위한 코드
# csvTestFile =  files.upload() 
# test = pd.read_csv(io.BytesIO(csvTestFile['csv_downloadfile (9).csv']))
# test
# proba = model.predict(test)
# idx = np.argmax(proba)
# proba

# 새 섹션

In [26]:
import tensorflowjs as tfjs #저장된 모델을 불러오기 위함
tfjs.converters.save_keras_model(model,artifacts_dir="./push_up_model.json")

'/content'